In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps 
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing 
%matplotlib inline

In [2]:
%%time
data = pd.read_csv('card transactions.csv')

CPU times: user 264 ms, sys: 55.5 ms, total: 320 ms
Wall time: 329 ms


In [3]:
#18 files, 96753 records
data.shape

(96753, 18)

In [4]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,1,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5142131721,1/1/10,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5142148452,1/1/10,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5142190439,1/1/10,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data = data.drop(columns=['Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96753 entries, 0 to 96752
Data columns (total 10 columns):
Recnum               96753 non-null int64
Cardnum              96753 non-null int64
Date                 96753 non-null object
Merchnum             93378 non-null object
Merch description    96753 non-null object
Merch state          95558 non-null object
Merch zip            92097 non-null float64
Transtype            96753 non-null object
Amount               96753 non-null float64
Fraud                96753 non-null int64
dtypes: float64(2), int64(3), object(5)
memory usage: 7.4+ MB


# filter P

In [7]:
data = data[data.Transtype == 'P']

In [8]:
data.shape

(96398, 10)

### remove outliner

In [9]:
data['Amount'].sort_values(ascending = False).head(10)

52714    3102045.53
47339      47900.00
59516      30372.46
80886      28392.84
89673      27218.00
1103       26910.00
89408      25805.35
69666      25000.00
10173      24990.00
46937      24946.90
Name: Amount, dtype: float64

In [10]:
data[data['Amount'] == 3102045.53]['Recnum']

52714    52715
Name: Recnum, dtype: int64

In [11]:
data = data[data.Recnum != 52715]

In [12]:
data['Amount'].max()

47900.0

# Filter out of time

In [13]:
from datetime import datetime

In [14]:
data['Date'] =  pd.to_datetime(data['Date'])

In [15]:
data['Date'].head()

0   2010-01-01
1   2010-01-01
2   2010-01-01
3   2010-01-01
4   2010-01-01
Name: Date, dtype: datetime64[ns]

In [16]:
data = data[data['Date'] < '2010-11-01']

In [17]:
data.shape

(83970, 10)

# 1. Fill State

In [18]:
data_notnull = data[data['Merch state'].notnull()]
state_zip=data_notnull.groupby("Merch zip")["Merch state"].apply(lambda x: sps.mode(x)[0][0]).reset_index()

data=data.merge(state_zip,on="Merch zip",how="left")

data["Merch state fill"]=data["Merch state_x"].fillna(data["Merch state_y"])

data["Merch state fill"]=data["Merch state_x"].fillna("TN")

data["Merch state fill"].isnull().sum()

/Users/kailinghung/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


0

# 2. Fill Merch zip

In [19]:
data["Merch zip"].isnull().sum()

3366

In [20]:
data["Merch zip"].describe()

count    80604.000000
mean     44639.600677
std      28225.631383
min          1.000000
25%      20884.000000
50%      38118.000000
75%      63103.000000
max      99999.000000
Name: Merch zip, dtype: float64

In [21]:
data["Merch zip"].value_counts().head()

38118.0    11142
63103.0     1432
8701.0      1131
22202.0     1123
60061.0     1079
Name: Merch zip, dtype: int64

In [22]:
data["Merch zip"].value_counts().tail()

45365.0    1
14604.0    1
74401.0    1
37205.0    1
5.0        1
Name: Merch zip, dtype: int64

## a. Fill ZIP by merchnum

In [23]:
data_notnull = data[data["Merchnum"].notnull()]
data_notnull = data_notnull[data_notnull["Merchnum"] != '0' ]
data_notnull = data_notnull[data_notnull["Merch zip"] != 0 ]
data_mernum = data_notnull.groupby("Merchnum")["Merch zip"].apply(lambda x: sps.mode(x)[0][0]).reset_index()
data_mernum.head()

,Merchnum,Merch zip
0,003100006NOT6,41011.0
1,004740006ABC6,82520.0
2,014430619 14,90502.0
3,014948837 52,92101.0
4,016560006NOT0,7073.0


In [24]:
data = data.merge(data_mernum, on =['Merchnum'], how = 'left')
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state_x,Merch zip_x,Transtype,Amount,Fraud,Merch state_y,Merch state fill,Merch zip_y
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,TN,TN,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,MA,MA,1803.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,MD,MD,20706.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,TN,TN,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,TN,TN,38118.0


In [25]:
data["Merch zip_y"]=data["Merch zip_y"].replace(0, np.nan)

In [26]:
data["Merch zip_y"].describe()

count    80340.000000
mean     44391.067015
std      28120.899131
min          1.000000
25%      20884.000000
50%      38118.000000
75%      63103.000000
max      99999.000000
Name: Merch zip_y, dtype: float64

In [27]:
data["Merch zip_y"].isna().value_counts()

False    80340
True      3630
Name: Merch zip_y, dtype: int64

In [28]:
data["Merch zip fill"]=data["Merch zip_x"].fillna(data["Merch zip_y"])

In [29]:
# still have 2093 null nalues
data["Merch zip fill"].isnull().sum()

2093

In [30]:
data = data.drop(columns=['Merch zip_x','Merch zip_y'])
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state_x,Transtype,Amount,Fraud,Merch state_y,Merch state fill,Merch zip fill
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,P,31.42,0,MA,MA,1803.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,P,178.49,0,MD,MD,20706.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,P,3.62,0,TN,TN,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0


## b. Fill ZIP by Merch description

In [31]:
data_des = data.groupby("Merch description")["Merch zip fill"].apply(lambda x: sps.mode(x)[0][0]).reset_index()
data_des.head()

,Merch description,Merch zip fill
0,#9 SOFTWARE,23663.0
1,(ISC)2 CERTIFICATION,1545.0
2,0000000000000000000000,98052.0
3,033007 KINKO'S,27713.0
4,05032 FLYING J,87105.0


In [32]:
data = data.merge(data_des, on =['Merch description'], how = 'left')
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state_x,Transtype,Amount,Fraud,Merch state_y,Merch state fill,Merch zip fill_x,Merch zip fill_y
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,P,31.42,0,MA,MA,1803.0,1803.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,P,178.49,0,MD,MD,20706.0,20706.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,P,3.62,0,TN,TN,38118.0,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0,38118.0


In [33]:
data["Merch zip fill_y"]=data["Merch zip fill_y"].replace(0, np.nan)

In [34]:
data["Merch zip fill_y"].describe()

count    82236.000000
mean     44271.751180
std      28300.583587
min          1.000000
25%      20874.000000
50%      38118.000000
75%      63103.000000
max      99999.000000
Name: Merch zip fill_y, dtype: float64

In [35]:
data["Merch zip fill"]=data["Merch zip fill_x"].fillna(data["Merch zip fill_y"])

In [36]:
data = data.drop(columns=['Merch zip fill_x','Merch zip fill_y'])
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state_x,Transtype,Amount,Fraud,Merch state_y,Merch state fill,Merch zip fill
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,P,31.42,0,MA,MA,1803.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,P,178.49,0,MD,MD,20706.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,P,3.62,0,TN,TN,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0


In [37]:
# still have 1734 null value
data["Merch zip fill"].isnull().sum()

1734

## c. Fill Zip by State

In [38]:
data_notnull1 = data[data["Merch zip fill"].notnull()]
data_state = data_notnull1.groupby("Merch state fill")["Merch zip fill"].apply(lambda x: sps.mode(x)[0][0]).reset_index()
data_state.head()

,Merch state fill,Merch zip fill
0,AK,99501.0
1,AL,35801.0
2,AR,72764.0
3,AZ,85285.0
4,CA,92656.0


In [39]:
data = data.merge(data_state, on =['Merch state fill'], how = 'left')
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state_x,Transtype,Amount,Fraud,Merch state_y,Merch state fill,Merch zip fill_x,Merch zip fill_y
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,P,31.42,0,MA,MA,1803.0,1730.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,P,178.49,0,MD,MD,20706.0,20763.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,P,3.62,0,TN,TN,38118.0,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0,38118.0


In [40]:
data["Merch zip fill_y"].describe()

count    83937.000000
mean     44948.400372
std      28269.275805
min       1730.000000
25%      22202.000000
50%      38118.000000
75%      63103.000000
max      99999.000000
Name: Merch zip fill_y, dtype: float64

In [41]:
data["Merch zip fill"]=data["Merch zip fill_x"].fillna(data["Merch zip fill_y"])

In [42]:
data["Merch zip fill"].isnull().sum()

33

In [43]:
data = data.drop(columns=['Merch zip fill_x','Merch zip fill_y'])
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state_x,Transtype,Amount,Fraud,Merch state_y,Merch state fill,Merch zip fill
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,P,31.42,0,MA,MA,1803.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,P,178.49,0,MD,MD,20706.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,P,3.62,0,TN,TN,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,P,3.62,0,TN,TN,38118.0


In [44]:
data["Merch zip fill"].describe()

count    83937.000000
mean     44302.685371
std      28275.350621
min          1.000000
25%      20877.000000
50%      38118.000000
75%      63103.000000
max      99999.000000
Name: Merch zip fill, dtype: float64

## d. Fill Zip by zipmode

In [45]:
data["Merch zip fill"].value_counts().head()

38118.0    11621
22202.0     1712
63103.0     1450
8701.0      1134
60061.0     1097
Name: Merch zip fill, dtype: int64

In [46]:
data["Merch zip fill"]=data["Merch zip fill"].replace(np.nan,38118.0)

In [47]:
## all merch zip filled
data["Merch zip fill"].isnull().sum()

0

In [48]:
data = data.drop(columns=['Merch state_x','Merch state_y'])

In [49]:
data.shape

(83970, 10)

In [50]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Transtype,Amount,Fraud,Merch state fill,Merch zip fill
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,P,3.62,0,TN,38118.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,P,31.42,0,MA,1803.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,P,178.49,0,MD,20706.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,P,3.62,0,TN,38118.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,P,3.62,0,TN,38118.0


In [51]:
data.to_csv("Filled In Missing Value.csv")